#### SELEÇÃO DO MELHOR MODELO PELO AIC - STEPWISE

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Cálculo do Critério de Informação de Akaike (AIC)

In [ ]:
def calculate_aic(model, X, y):
    n = len(y)
    k = X.shape[1] + 1  # número de parâmetros, incluindo o intercepto
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    aic = n * np.log(mse) + 2 * k
    return aic

Realizando a seleção de variáveis STEPWISE bidireicional (forward-backward stepwise selection)

In [3]:
def forward_backward_stepwise_selection(X, y):
    selected_features = [] # lista vazia que armazenará as variáveis selecionadas
    stepwise_model = LinearRegression() # modelo de regressão a ser usado
    best_aic = float('inf') # variável que armazenará o melhor AIC
    #(lembrando que quanto menor o AIC, melhor o modelo)
    
    # Passo Forward
    while True: #Este loop continua até que não haja mais variáveis para serem testadas.
        features_to_test = [feature for feature in X.columns if feature not in selected_features]
        if not features_to_test: #Lista das variáveis que ainda não foram selecionadas.
            break
        
        best_feature = None
        best_model = None
        best_aic_with_new_feature = float('inf')
        
        #Para cada variável em features_to_test, o modelo linear é ajustado utilizando essa variável em 
        #conjunto com as variáveis já selecionadas.
        for feature in features_to_test:
            # Ajusta o modelo com a nova variável adicionada
            features_selected = selected_features + [feature]
            stepwise_model.fit(X[features_selected], y)
            # Calcula o AIC
            aic = calculate_aic(stepwise_model, X[features_selected], y)
            # Seleciona a variável com menor AIC
            if aic < best_aic_with_new_feature:
                best_aic_with_new_feature = aic
                best_feature = feature
                best_model = stepwise_model
                
        if best_aic_with_new_feature >= best_aic:
            break
        
        selected_features.append(best_feature)
        best_aic = best_aic_with_new_feature
    
    # Passo Backward
    while True:
        if not selected_features:
            break
        
        worst_feature = None
        worst_model = None
        best_aic_without_feature = float('inf')
        
        for feature in selected_features:
            # Ajusta o modelo sem a variável atual
            features_removed = [f for f in selected_features if f != feature]
            stepwise_model.fit(X[features_removed], y)
            # Calcula o AIC
            aic = calculate_aic(stepwise_model, X[features_removed], y)
            # Seleciona a variável com menor AIC
            if aic < best_aic_without_feature:
                best_aic_without_feature = aic
                worst_feature = feature
                worst_model = stepwise_model
        
        if best_aic_without_feature >= best_aic:
            break
        
        selected_features.remove(worst_feature)
        best_aic = best_aic_without_feature
    
    return selected_features

In [2]:
# Load the data
df = pd.read_excel("diabetes_dataset.xlsx")

# Define the dependent and independent variables
y = df["progressao"]
X = df.drop("progressao", axis=1)

# Inicializa o modelo de regressão linear
stepwise_model = LinearRegression()

# Realiza a seleção de variáveis
selected_features = forward_backward_stepwise_selection(X, y)

# Exibe as variáveis selecionadas
print("Variáveis Selecionadas:", selected_features)

Variáveis Selecionadas: ['bmi', 's5', 'bp', 's1', 'sex', 's2']
